In [ ]:
# Import libraries/packages
import pandas as pd
from pathlib import Path
import numpy as np

In [ ]:
# Load cleaned and anonymized data
ROOT = Path.cwd().parent
fuel_data_raw = pd.read_csv(ROOT / 'data/raw/fuel_data_raw_anon.csv')

#### Trailer Configuration Dataset

In [ ]:
# Parse delivery dates and create YearMonth column
fuel_data_raw['Delivery_Date'] = pd.to_datetime(fuel_data_raw['Delivery_Date'])
fuel_data_raw['YearMonth'] = fuel_data_raw['Delivery_Date'].dt.to_period('M')

# Copy entire dataset for further filtering
filtered_data = fuel_data_raw.copy()

# Prepare truck configuration using Trailer and Tank Capacity
truck_sample = filtered_data[['Trailer', 'Truck_Tank_Capacity_Liters']].drop_duplicates()

def assign_type(cap):
    if cap < 18000:
        return 'Small'
    elif cap < 30000:
        return 'Medium'
    else:
        return 'Large'

truck_sample['Truck_Type'] = truck_sample['Truck_Tank_Capacity_Liters'].apply(assign_type)

# Assign max trips and distance limits per truck type
truck_sample['Max_Trips_Per_Day'] = truck_sample['Truck_Type'].map({
    'Small': 4,
    'Medium': 5,
    'Large': 6
})

truck_sample['Distance_Limit_Per_Day'] = truck_sample['Truck_Type'].map({
    'Small': 200,
    'Medium': 250,
    'Large': 350
})

# Calculate trailer availability percent
total_days = (filtered_data['Delivery_Date'].max() - filtered_data['Delivery_Date'].min()).days + 1
trailer_day_counts = (
    filtered_data
    .groupby('Trailer')['Delivery_Date']
    .nunique()
    .reset_index(name='Active_Days')
)
trailer_day_counts['Availability_Percent'] = (trailer_day_counts['Active_Days'] / total_days) * 100

truck_sample = pd.merge(truck_sample, trailer_day_counts, on='Trailer', how='left')
truck_sample['Availability_Percent'] = truck_sample['Availability_Percent'].fillna(0).round(2)

In [ ]:
# Save truck configuration CSV
truck_csv_path = ROOT / 'data/processed/truck_config_julia.csv'
truck_sample.to_csv(truck_csv_path, index=False)

#### Station Demand Dataset

In [ ]:
# Filter only by Origin_ID == 'Source_2' and rename to Source_Depot
filtered_data = filtered_data[filtered_data['Origin_ID'] == 'Source_2'].copy()
filtered_data['Origin_ID'] = 'Source_Depot'

# Group by Destination, Product, and Month to get monthly demand
monthly_demand = (
    filtered_data
    .groupby(['Destination_ID', 'Product', 'YearMonth'])['Delivered_Volume_Liters']
    .sum()
    .reset_index()
)

# Calculate average monthly demand per station and product
avg_monthly_demand = (
    monthly_demand
    .groupby(['Destination_ID', 'Product'])['Delivered_Volume_Liters']
    .mean()
    .reset_index()
    .rename(columns={'Delivered_Volume_Liters': 'Monthly_Demand_Liters'})
)

# Extract and apply 80% buffer to product storage capacity
storage_capacity = (
    filtered_data[['Destination_ID', 'Product', 'Product_Storage_Capacity_Liters']]
    .drop_duplicates()
    .copy()
)
storage_capacity['Product_Storage_Capacity_Liters'] *= 0.80

# Merge demand and storage capacity
station_demand = pd.merge(avg_monthly_demand, storage_capacity, on=['Destination_ID', 'Product'], how='left')

# Add station distance
station_distances = filtered_data[['Destination_ID', 'Distance_km']].drop_duplicates()
station_demand = pd.merge(station_demand, station_distances, on='Destination_ID', how='left')

# Compute estimated number of refills per month (ceiled)
station_demand['Estimated_Refills'] = np.ceil(
    station_demand['Monthly_Demand_Liters'] / station_demand['Product_Storage_Capacity_Liters']
).astype(int)

In [ ]:
# Save updated station demand CSV
station_csv_path = ROOT / 'data/processed/station_demand_julia.csv'
station_demand.to_csv(station_csv_path, index=False)